In [1]:
# colab has an older version of beautifulsoup by default
# here we upgrade it
# if you are working on your own computer, you can probably comment this step out and skip it
!pip install --upgrade beautifulsoup4

In [2]:
# 1. import libraries
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re

In [3]:
# 2. find url and store it in a variable
url = "https://en.wikipedia.org/wiki/Berlin"

In [4]:
# 3. download html with a get request
response = requests.get(url)
response.status_code # 200 status code means OK!

200

In [5]:
# 4.1. parse html (create the 'soup')
soup = BeautifulSoup(response.content, "html.parser")
# 4.2. check that the html code looks like it should
# soup

In [6]:
# 5. retrieve/extract the desired info (here, you'll paste the "Selector" you copied before to get the element that belongs to the top movie)

# let's first try to get the name of the city
# by copying the selector we can see that it has the id firstHeading (it also has a class by the same name!)
soup.select("#firstHeading")

[<h1 class="firstHeading mw-first-heading" id="firstHeading"><span class="mw-page-title-main">Berlin</span></h1>]

In [7]:
soup.select("#firstHeading")[0].get_text()

'Berlin'

In [8]:
# Let's use this class, infobox-data, to target the information country
soup.select(".infobox-data")[0].get_text()

'Germany'

In [ ]:
#soup.select(".infobox-data")[0].get_text()

Now we just carry on exploring the html, finding classes, ids, and selectors to target the information we need. Hopefully these classes and selectors will be universal across all cities on wikipedia, but it is likely that they will change in a few places, and we will have to try to make our code robust to this

In [26]:
def recreate_wiki(cities):
  # empty list that will be filled with one dictionary of information per city
  list_for_df = []
  
  # begin a for loop to create a dictionary of information for each city
  for city in cities:
    # we can use the universal nature of wikipedias urls to our advantage here
    # all of the urls are the same besides the city name
    url = f'https://en.wikipedia.org/wiki/{city}'

    # here we make our soup for the city
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'html.parser')

    # here we initialise our empty dictionary for the city
    response_dict = {}

    # here we fill the dictionary with information using the ids, classes, and selectors that we found in the html
    response_dict['city'] = soup.select(".firstHeading")[0].get_text()
    response_dict['country'] = soup.select(".infobox-data")[0].get_text()
    response_dict['latitude'] = soup.select(".latitude")[0].get_text()
    response_dict['longitude'] = soup.select(".longitude")[0].get_text()
    # not all of the wikipedia pages contain elevation, look at Hamburg
    # the if clause means that our code can continue and won't stop at this hurdle
    if soup.select_one('.infobox-label:-soup-contains("Elevation")'):
      response_dict['elevation'] = soup.select_one('.infobox-label:-soup-contains("Elevation")').find_next(class_='infobox-data').get_text()
    response_dict['website'] = soup.select_one('.infobox-label:-soup-contains("Website")').find_next(class_='infobox-data').get_text()
    if soup.select_one('th.infobox-header:-soup-contains("Population")'):
        response_dict['population'] = soup.select_one('th.infobox-header:-soup-contains("Population")').parent.find_next_sibling().find(text=re.compile(r'\d+'))
    
    # add our dictionary for the city to list_for_df
    list_for_df.append(response_dict)
  
  # make the DataFrame
  cities_df = pd.DataFrame(list_for_df)

  # fixing latitude
  cities_df['latitude'] = cities_df['latitude'].str.split('″').str[0].str.replace('°', '.', regex=False).str.replace('′', '', regex=False).str.replace('N', '', regex=False)
  # fixing longitude
  cities_df['longitude'] = cities_df['longitude'].str.split('″').str[0].str.replace('°', '.', regex=False).str.replace('′', '', regex=False).str.replace('E', '', regex=False)
  # fixing elevation
  cities_df.insert(4, 'elevation_in_meters', cities_df['elevation'].str.split('m').str[0].str.strip())
  cities_df['population'] = cities_df['population'].str.replace(',', '', regex=False)

  # return the DataFrame
  return cities_df

list_of_cities = ['Berlin', 'Hamburg', 'London', 'Manchester', 'Barcelona']
cities = recreate_wiki(list_of_cities)
list_of_cities = ['Frankfurt', 'Berlin', 'Cologne', 'Munich', 'Hamburg']
cities = recreate_wiki(list_of_cities)
cities_data = cities[['city','country','latitude','longitude','population']]
cities_e_bike = cities_data.copy() 
cities_e_bike['latitude'] = cities_e_bike['latitude'].astype(float)
cities_e_bike['longitude'] = cities_e_bike['longitude'].astype(float)
cities_e_bike['population'] = pd.to_numeric(cities_e_bike['population'],errors='coerce')
cities_e_bike

/var/folders/sv/djrh3svn0hl9fv9d7jw8xwl40000gn/T/ipykernel_46407/4234743330.py:29: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  response_dict['population'] = soup.select_one('th.infobox-header:-soup-contains("Population")').parent.find_next_sibling().find(text=re.compile(r'\d+'))
/var/folders/sv/djrh3svn0hl9fv9d7jw8xwl40000gn/T/ipykernel_46407/4234743330.py:29: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  response_dict['population'] = soup.select_one('th.infobox-header:-soup-contains("Population")').parent.find_next_sibling().find(text=re.compile(r'\d+'))


,city,country,latitude,longitude,population
0,Frankfurt,Germany,50.0638,8.4056,759224
1,Berlin,Germany,52.3112,13.2418,3677472
2,Cologne,Germany,50.5611,6.5710,1073096
3,Munich,Germany,48.0815,11.3430,1487708
4,Hamburg,Germany,53.3300,10.0000,1906411
